In [3]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("../../../bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
stats = importr('stats')

MHC-II only ORs

In [7]:
OR = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/OR_clean/tissue/class_ii/cancer.thresh_2.txt').reset_index()
OR.columns = ['Tumor Type', 'OR', 'Low 95% CI', 'High 95% CI', 'P-value']

In [8]:
OR

,Tumor Type,OR,Low 95% CI,High 95% CI,P-value
0,GBM,1.134945,1.028262,1.252696,1.195881e-02
1,OV,1.392231,1.175481,1.648948,1.268349e-04
2,LUAD,1.088369,1.007708,1.175487,3.112749e-02
3,LUSC,1.177393,1.076691,1.287515,3.438471e-04
4,PRAD,0.833787,0.738265,0.941670,3.410183e-03
5,BLCA,1.311011,1.193558,1.440022,1.559647e-08
6,PAAD,1.158787,1.034452,1.298066,1.093030e-02
7,LIHC,1.072802,0.945574,1.217149,2.752285e-01
8,STAD,1.084404,1.001311,1.174393,4.634903e-02
9,SKCM,1.228322,1.152650,1.308963,2.311788e-10


In [9]:
OR = OR[OR['Tumor Type'].isin(['THCA', 'LGG', 'GBM', 'SKCM', 'BLCA', 'OV', 'LUSC', 'PAAD',
                               'COAD', 'HNSC', 'STAD', 'LUAD', 'BRCA', 'LIHC', 'READ',
                               'UCEC', 'UCS', 'PRAD'])]

In [10]:
OR['Adj-P'] = stats.p_adjust(FloatVector(OR['P-value']), method = 'BH')

In [11]:
def significance(x):
    if x < 0.05:
        return "*"
    else:
        return ""
OR['Significance'] = OR['Adj-P'].apply(significance)

In [15]:
len(OR)

18

In [18]:
OR.sort_values('OR', ascending=False)

,Tumor Type,OR,Low 95% CI,High 95% CI,P-value,Adj-P,Significance
10,THCA,2.631089,2.287174,3.026717,9.617690e-42,5.770614e-41,*
16,LGG,1.920088,1.761921,2.092454,4.874776e-50,4.387299e-49,*
1,OV,1.392231,1.175481,1.648948,1.268349e-04,3.805047e-04,*
5,BLCA,1.311011,1.193558,1.440022,1.559647e-08,5.614731e-08,*
9,SKCM,1.228322,1.152650,1.308963,2.311788e-10,1.040304e-09,*
3,LUSC,1.177393,1.076691,1.287515,3.438471e-04,7.736560e-04,*
12,HNSC,1.169428,1.078221,1.268349,1.581827e-04,4.067555e-04,*
6,PAAD,1.158787,1.034452,1.298066,1.093030e-02,1.956896e-02,*
0,GBM,1.134945,1.028262,1.252696,1.195881e-02,1.956896e-02,*
2,LUAD,1.088369,1.007708,1.175487,3.112749e-02,4.669124e-02,*


In [12]:
OR.to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/tables/MHCII.tissue.OR.csv', index=False)

Output to file

In [16]:
from pandas import ExcelWriter
writer = ExcelWriter('/cellar/users/ramarty/Data/hla_ii/generated_data/tables/tumor_type_OR.xlsx')
OR.sort_values('OR').to_excel(writer,'MHC-II Only Model', index=False)
writer.save()

Compare to MHC-I

In [20]:
OR = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/OR_clean/tissue/class_i/cancer.thresh_2.txt').reset_index()
OR.columns = ['Tumor Type', 'OR', 'Low 95% CI', 'High 95% CI', 'P-value']

In [21]:
OR

,Tumor Type,OR,Low 95% CI,High 95% CI,P-value
0,GBM,1.088406,0.994890,1.190712,6.456883e-02
1,OV,1.021289,0.895452,1.164811,7.535183e-01
2,LUAD,1.147654,1.066764,1.234678,2.215259e-04
3,LUSC,0.943063,0.874485,1.017019,1.280383e-01
4,PRAD,1.165991,1.008455,1.348137,3.810863e-02
5,BLCA,1.174856,1.085390,1.271697,6.673318e-05
6,PAAD,1.262710,1.135109,1.404654,1.773803e-05
7,LIHC,1.099287,0.975144,1.239235,1.215484e-01
8,STAD,1.091680,1.013334,1.176083,2.096467e-02
9,SKCM,1.129718,1.069098,1.193775,1.461285e-05
